In [5]:
import pandas as pd
import requests
import io

# This is the special URL you created that exports the Google Sheet as a CSV.
# これは、GoogleスプレッドシートをCSVとしてエクスポートするためにあなたが作成した特別なURLです。
csv_url = "https://docs.google.com/spreadsheets/d/1OscrXVHgOn75NrsrUxJfitmcPBHRj6xlnRcsq9jXo2g/export?format=csv&gid=590671728"

print("--- Method 1: The Direct Pandas Way (Easiest) ---")
# --- 方法1：Pandasで直接読み込む方法（最も簡単）---

try:
    # Pandas can read a CSV directly from a URL.
    # PandasはURLから直接CSVを読み込むことができます。
    df_method1 = pd.read_csv(csv_url)
    
    # Print the first 5 rows to check the data.
    # 最初の5行を印刷してデータを確認します。
    print("Successfully loaded data using Method 1:")
    print(df_method1.head())

except Exception as e:
    print(f"Method 1 failed: {e}")

print("\n" + "="*50 + "\n")

print("--- Method 2: The 'requests' Way (More control) ---")
# --- 方法2：「requests」ライブラリを使う方法（より制御が効く）---
# This method is better if you want to check if the download was successful
# or handle errors.
# この方法は、ダウンロードが成功したか確認したり、エラーを処理したりしたい場合に優れています。

try:
    # First, download the content from the URL.
    # まず、URLからコンテンツをダウンロードします。
    response = requests.get(csv_url)
    
    # Check if the download was successful (Status code 200 means OK).
    # ダウンロードが成功したか確認します（ステータスコード 200 は OK を意味します）。
    if response.status_code == 200:
        print("Download successful (Status 200)")
        
        # The content is in text format. We need to convert it into a
        # file-like object that pandas can read.
        # コンテンツはテキスト形式です。これをpandasが読み取れる
        # ファイルのようなオブジェクトに変換する必要があります。
        csv_content = response.content.decode('utf-8')
        csv_file_like_object = io.StringIO(csv_content)
        
        # Now, read this "file" with pandas.
        # この「ファイル」をpandasで読み込みます。
        df_method2 = pd.read_csv(csv_file_like_object)
        
        print("Successfully loaded data using Method 2:")
        print(df_method2.head())
        
        # Now you have the data in a DataFrame (df_method2)
        # これで、データがDataFrame (df_method2) に格納されました。
        # You can start your analysis:
        # ここから分析を開始できます：
        # print(df_method2.describe())
        
    else:
        # If the status code isn't 200, something went wrong.
        # ステータスコードが200でない場合、何かがうまくいきませんでした。
        print(f"Error: Download failed with status code {response.status_code}")

except Exception as e:
    print(f"Method 2 failed: {e}")
    

--- Method 1: The Direct Pandas Way (Easiest) ---
Successfully loaded data using Method 1:
  Parking Occupancy : Fall 2025 Week 0 Unnamed: 1         Unnamed: 2  \
0         Thursday, September 25, 2025        NaN                NaN   
1                           Space Type        NaN           Location   
2                                    A  Structure               8980   
3                                    A        NaN             Athena   
4                                    A        NaN  Campus Point East   

         Unnamed: 3 Unnamed: 4 Unnamed: 5 Unnamed: 6  
0  Available Spaces        NaN        NaN        NaN  
1           8:00 AM   10:00 AM   12:00 PM    2:00 PM  
2                25         23         19         20  
3               217        140        108        112  
4                94         23         13         17  


--- Method 2: The 'requests' Way (More control) ---
Download successful (Status 200)
Successfully loaded data using Method 2:
  Parking Occupanc

In [2]:
df_method1

,Parking Occupancy : Fall 2025 Week 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,"Thursday, September 25, 2025",NaN,NaN,Available Spaces,NaN,NaN,NaN
1,Space Type,NaN,Location,8:00 AM,10:00 AM,12:00 PM,2:00 PM
2,A,Structure,8980,25,23,19,20
3,A,NaN,Athena,217,140,108,112
4,A,NaN,Campus Point East,94,23,13,17
...,...,...,...,...,...,...,...
107,Visitor,NaN,P510,4,2,0,0
108,Visitor,NaN,P602,7,2,0,1
109,Visitor,NaN,P603,22,12,5,12
110,Visitor,NaN,P703,9,10,8,9


In [6]:
import pandas as pd
import requests
import io
import time
from bs4 import BeautifulSoup # You'll need to install this: pip install beautifulsoup4
import re

# --- 1. Scrape the main page to find all the Google Sheet links ---
# --- 1. メインページをスクレイピングして、すべてのGoogleスプレッドシートのリンクを見つけます ---

# The main page we want to parse
# 解析したいメインページ
main_url = "https://transportation.ucsd.edu/commute/permits/availability.html"

# This dictionary will hold all our data
# この辞書がすべてのデータを保持します
all_dataframes = {}

print(f"Fetching main page: {main_url}")
# メインページを取得中: {main_url}

try:
    # Get the HTML content of the main page
    # メインページのHTMLコンテンツを取得します
    main_page_response = requests.get(main_url)
    
    if main_page_response.status_code == 200:
        print("Successfully fetched main page.")
        # メインページの取得に成功しました。
        
        # Parse the HTML with BeautifulSoup
        # BeautifulSoupでHTMLを解析します
        soup = BeautifulSoup(main_page_response.content, 'html.parser')
        
        # Find all <iframe> tags in the HTML
        # HTML内のすべての <iframe> タグを見つけます
        iframes = soup.find_all('iframe')
        
        print(f"Found {len(iframes)} <iframe> tags. Filtering for Google Sheets...")
        # {len(iframes)} 個の <iframe> タグを見つけました。Googleスプレッドシートでフィルタリング中...

        # Store the unique CSV links we find
        # 見つかったユニークなCSVリンクを保存します
        csv_links = {}

        for frame in iframes:
            # Get the 'src' attribute from the iframe tag
            # iframeタグから 'src' 属性を取得します
            src_link = frame.get('src')
            
            # Check if it's a Google Sheet /pubhtml link
            # これがGoogleスプレッドシートの /pubhtml リンクか確認します
            if src_link and 'docs.google.com/spreadsheets' in src_link and '/pubhtml' in src_link:
                
                # --- 2. Convert the /pubhtml link to a /export?format=csv link ---
                # --- 2. /pubhtml リンクを /export?format=csv リンクに変換します ---
                
                # We need to extract the Sheet ID and the GID
                # シートIDとGIDを抽出する必要があります
                
                # Extract Sheet ID
                # e.g., .../d/1OscrXVHgOn75NrsrUxJfitmcPBHRj6xlnRcsq9jXo2g/pubhtml...
                sheet_id_match = re.search(r'/d/([a-zA-Z0-9-_]+)', src_link)
                
                # Extract GID
                # e.g., ...gid=590671728...
                gid_match = re.search(r'gid=([0-9]+)', src_link)
                
                if sheet_id_match and gid_match:
                    sheet_id = sheet_id_match.group(1)
                    gid = gid_match.group(1)
                    
                    # Create the unique CSV download link
                    # ユニークなCSVダウンロードリンクを作成します
                    csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid}"
                    
                    # Add it to our list, using the GID as a key to avoid duplicates
                    # 重複を避けるためにGIDをキーとしてリストに追加します
                    csv_links[gid] = csv_url

        print(f"Found {len(csv_links)} unique Google Sheet tabs to download.")
        # ダウンロードするユニークなGoogleスプレッドシートのタブを {len(csv_links)} 個見つけました。

        # --- 3. Loop and Download each CSV ---
        # --- 3. 各CSVをループしてダウンロードします ---
        for gid, download_url in csv_links.items():
            
            print(f"\nDownloading data for GID: {gid}")
            # GID: {gid} のデータをダウンロード中
            
            try:
                response = requests.get(download_url)
                if response.status_code == 200:
                    csv_content = response.content.decode('utf-8')
                    csv_file = io.StringIO(csv_content)
                    
                    df = pd.read_csv(csv_file)
                    
                    # We'll store the data using the GID as the key
                    # GIDをキーとしてデータを保存します
                    all_dataframes[gid] = df
                    
                    print(f"  [SUCCESS] GID '{gid}' loaded with {len(df)} rows.")
                    #   [成功] GID '{gid}' に {len(df)} 行が読み込まれました。
                else:
                    print(f"  [FAILED] Could not download GID {gid}. Status: {response.status_code}")
                    #   [失敗] GID {gid} をダウンロードできませんでした。ステータス: {response.status_code}
            
            except Exception as e:
                print(f"  [FAILED] An error occurred for GID {gid}: {e}")
                #   [失敗] GID {gid} でエラーが発生しました: {e}
            
            time.sleep(1) # Be polite

    else:
        print(f"Failed to fetch main page. Status code: {main_page_response.status_code}")
        # メインページの取得に失敗しました。ステータスコード: {main_page_response.status_code}

except Exception as e:
    print(f"An error occurred: {e}")
    # エラーが発生しました: {e}

print("\n" + "="*50)
print("Scraping complete.")
# スクレイピングが完了しました。
print(f"Successfully downloaded {len(all_dataframes)} tables.")
# {len(all_dataframes)} 個のテーブルを正常にダウンロードしました。

# --- 4. NOW YOU CAN USE THE DATA ---
# --- 4. これでデータを使用できます ---
if all_dataframes:
    first_gid = list(all_dataframes.keys())[0]
    print(f"\n--- Data from GID '{first_gid}' ---")
    # GID '{first_gid}' からのデータ ---
    print(all_dataframes[first_gid].head())

Fetching main page: https://transportation.ucsd.edu/commute/permits/availability.html
Successfully fetched main page.
Found 12 <iframe> tags. Filtering for Google Sheets...
Found 12 unique Google Sheet tabs to download.

  [SUCCESS] GID '590671728' loaded with 112 rows.

  [SUCCESS] GID '1596814157' loaded with 112 rows.

  [SUCCESS] GID '0' loaded with 113 rows.

  [SUCCESS] GID '1732410273' loaded with 113 rows.

  [SUCCESS] GID '980526681' loaded with 110 rows.

  [SUCCESS] GID '1691861364' loaded with 110 rows.

  [SUCCESS] GID '1701491299' loaded with 110 rows.

  [SUCCESS] GID '1997138837' loaded with 110 rows.

  [SUCCESS] GID '1843964475' loaded with 110 rows.

  [SUCCESS] GID '871989841' loaded with 110 rows.

  [FAILED] Could not download GID 1378029010. Status: 404

  [FAILED] Could not download GID 293064621. Status: 404

Scraping complete.
Successfully downloaded 10 tables.

--- Data from GID '590671728' ---
  Parking Occupancy : Fall 2025 Week 0 Unnamed: 1         Unnamed

In [8]:
all_dataframes['590671728']

,Parking Occupancy : Fall 2025 Week 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,"Thursday, September 25, 2025",NaN,NaN,Available Spaces,NaN,NaN,NaN
1,Space Type,NaN,Location,8:00 AM,10:00 AM,12:00 PM,2:00 PM
2,A,Structure,8980,25,23,19,20
3,A,NaN,Athena,217,140,108,112
4,A,NaN,Campus Point East,94,23,13,17
...,...,...,...,...,...,...,...
107,Visitor,NaN,P510,4,2,0,0
108,Visitor,NaN,P602,7,2,0,1
109,Visitor,NaN,P603,22,12,5,12
110,Visitor,NaN,P703,9,10,8,9


In [10]:
import pandas as pd
import requests
import io
import re

# --- 1. Download the single raw DataFrame for testing ---
# --- 1. テスト用の単一の生DataFrameをダウンロード ---

target_gid = '590671728'
sheet_id = '1OscrXVHgOn75NrsrUxJfitmcPBHRj6xlnRcsq9jXo2g'
download_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={target_gid}"

print(f"Downloading raw data for GID: {target_gid}...")
# GID: {target_gid} の生データをダウンロード中...

try:
    response = requests.get(download_url)
    if response.status_code == 200:
        csv_content = response.content.decode('utf-8')
        csv_file = io.StringIO(csv_content)
        
        # Read with no header, this is our raw, messy data
        # ヘッダーなしで読み込みます。これが生の、整理されていないデータです
        raw_df = pd.read_csv(csv_file, header=None)
        
        print("Raw data downloaded. Starting interactive cleaning...")
        # 生データをダウンロードしました。対話型クリーニングを開始します...

        # --- 2. Start Interactive Cleaning ---
        # --- 2. 対話型クリーニングの開始 ---

        # --- Step 2a: Find Date and Time ---
        # --- ステップ2a: 日付と時刻の検索 ---
        date_str = None
        time_str = None
        header_row_index = None

        df_string_dump = raw_df.to_string()
        date_match = re.search(r'(\w+,\s\w+\s\d+,\s\d{4})', df_string_dump)
        if date_match:
            date_str = date_match.group(1)
            print(f"  Found Date: {date_str}")
            #   日付が見つかりました: {date_str}

        time_match = re.search(r'(\d{1,2}:\d{2}\s[AP]M)', df_string_dump)
        if time_match:
            time_str = time_match.group(1)
            print(f"  Found Time: {time_str}")
            #   時刻が見つかりました: {time_str}

        # --- Step 2b: Find Header Row ---
        # --- ステップ2b: ヘッダー行の検索 ---
        for i, row in raw_df.iterrows():
            if 'Location' in row.values:
                header_row_index = i
                print(f"  Found Header at row index: {header_row_index}")
                #   ヘッダー行をインデックス: {header_row_index} で見つけました
                break
        
        if header_row_index is None:
            raise ValueError("Could not find header row with 'Location'")
            # 'Location' を含むヘッダー行が見つかりませんでした

        # --- Step 2c: Create Clean DataFrame ---
        # --- ステップ2c: クリーンなDataFrameの作成 ---
        new_header = raw_df.iloc[header_row_index]
        clean_df = raw_df[header_row_index + 1:].copy()
        clean_df.columns = new_header
        print("  Set new header.")
        #   新しいヘッダーを設定しました。

        # --- Step 2d: Clean Column Names ---
        # --- ステップ2d: カラム名のクリーンアップ ---
        
        # *** THIS IS THE FIX: ***
        # *** これが修正点です： ***
        # Force all column names to be strings *before* using .str
        # .str を使う*前*に、すべてのカラム名を文字列に強制します
        clean_df.columns = clean_df.columns.astype(str)
        
        clean_df.columns = clean_df.columns.str.strip()
        clean_df.rename(columns=lambda x: re.sub(r'\s\d{1,2}:\d{2}\s[AP]M', '', x), inplace=True)
        clean_df.columns = clean_df.columns.str.replace(' ', '_')
        print("  Cleaned column names.")
        #   カラム名をクリーンアップしました。

        # --- Step 2e: Add Metadata Columns ---
        # --- ステップ2e: メタデータカラムの追加 ---
        clean_df['Date'] = date_str
        clean_df['Time_of_Reading'] = time_str
        clean_df['Source_GID'] = target_gid
        print("  Added metadata columns.")
        #   メタデータカラムを追加しました。

        # --- Step 2f: Drop Empty Rows ---
        # --- ステップ2f: 空の行の削除 ---
        clean_df.dropna(subset=['Location'], inplace=True)
        print("  Dropped empty rows.")
        #   空の行を削除しました。

        # --- Step 2g: Final Polish (Type Conversion) ---
        # --- ステップ2g: 最終仕上げ（型変換） ---
        if 'Available_Spaces' in clean_df.columns:
            clean_df['Available_Spaces'] = pd.to_numeric(clean_df['Available_Spaces'], errors='coerce')
        clean_df['Date'] = pd.to_datetime(clean_df['Date'], errors='coerce')
        print("  Converted data types.")
        #   データ型を変換しました。

        # --- 3. Show Final Result ---
        # --- 3. 最終結果の表示 ---
        print("\n" + "="*50)
        print("--- FINAL CLEANED DATA (First 10 Rows) ---")
        # --- 最終クリーンアップデータ（最初の10行）---
        print(clean_df.head(10))
        
        print("\n--- FINAL DATA INFO ---")
        # --- 最終データ情報 ---
        clean_df.info()

    else:
        print(f"Failed to download. Status code: {response.status_code}")
        # ダウンロードに失敗しました。ステータスコード: {response.status_code}

except Exception as e:
    print(f"An error occurred: {e}")
    # エラーが発生しました: {e}

Raw data downloaded. Starting interactive cleaning...
  Found Date: Thursday, September 25, 2025
  Found Time: 8:00 AM
  Found Header at row index: 2
  Set new header.
  Cleaned column names.
  Added metadata columns.
  Dropped empty rows.
  Converted data types.

--- FINAL CLEANED DATA (First 10 Rows) ---
2  Space_Type        nan           Location 8:00_AM 10:00_AM 12:00_PM 2:00_PM  \
3           A  Structure               8980      25       23       19      20   
4           A        NaN             Athena     217      140      108     112   
5           A        NaN  Campus Point East      94       23       13      17   
6           A        NaN  Campus Point West       1        0        0       1   
7           A        NaN             Gilman     326      120       44      45   
8           A        NaN            Hopkins     110       96       72      83   
9           A        NaN             Pangea      57       44       34      37   
10          A        NaN           Scholars 

In [11]:
clean_df

2,Space_Type,nan,Location,8:00_AM,10:00_AM,12:00_PM,2:00_PM,Date,Time_of_Reading,Source_GID
3,A,Structure,8980,25,23,19,20,2025-09-25,8:00 AM,590671728
4,A,NaN,Athena,217,140,108,112,2025-09-25,8:00 AM,590671728
5,A,NaN,Campus Point East,94,23,13,17,2025-09-25,8:00 AM,590671728
6,A,NaN,Campus Point West,1,0,0,1,2025-09-25,8:00 AM,590671728
7,A,NaN,Gilman,326,120,44,45,2025-09-25,8:00 AM,590671728
...,...,...,...,...,...,...,...,...,...,...
108,Visitor,NaN,P510,4,2,0,0,2025-09-25,8:00 AM,590671728
109,Visitor,NaN,P602,7,2,0,1,2025-09-25,8:00 AM,590671728
110,Visitor,NaN,P603,22,12,5,12,2025-09-25,8:00 AM,590671728
111,Visitor,NaN,P703,9,10,8,9,2025-09-25,8:00 AM,590671728


In [7]:
all_dataframes.keys()

dict_keys(['590671728', '1596814157', '0', '1732410273', '980526681', '1691861364', '1701491299', '1997138837', '1843964475', '871989841'])

In [ ]:
def clean_and_restructure_data(df, gid):
    """
    Cleans a single raw DataFrame from the Google Sheet.
    - Finds and extracts the date and time.
    - Finds the real header row.
    - Returns a cleaned DataFrame.

    Googleスプレッドシートから取得した生のDataFrameを1つクリーンアップします。
    - 日付と時刻を見つけて抽出します。
    - 実際のヘッダー行を見つけます。
    - クリーンアップされたDataFrameを返します。
    """
    print(f"--- Cleaning GID {gid} ---")
    # --- GID {gid} をクリーニング中 ---
    
    # --- 1. Find Date and Time ---
    # --- 1. 日付と時刻の検索 ---
    date_str = None
    time_str = None
    header_row_index = None
    
    # Convert entire DataFrame to string to search for patterns
    # パターン検索のためにDataFrame全体を文字列に変換します
    df_string_dump = df.to_string()
    
    # Regex to find the date (e.g., "Thursday, September 25, 2025")
    # 日付を見つけるための正規表現（例：「Thursday, September 25, 2025」）
    date_match = re.search(r'(\w+,\s\w+\s\d+,\s\d{4})', df_string_dump)
    if date_match:
        date_str = date_match.group(1)
        print(f"  Found Date: {date_str}")
        #   日付が見つかりました: {date_str}

    # Regex to find the time (e.g., "8:00 AM")
    # 時刻を見つけるための正規表現（例：「8:00 AM」）
    time_match = re.search(r'(\d{1,2}:\d{2}\s[AP]M)', df_string_dump)
    if time_match:
        time_str = time_match.group(1)
        print(f"  Found Time: {time_str}")
        #   時刻が見つかりました: {time_str}

    # --- 2. Find the actual header row ---
    # --- 2. 実際のヘッダー行の検索 ---
    for i, row in df.iterrows():
        if 'Location' in row.values:
            header_row_index = i
            print(f"  Found Header at row index: {header_row_index}")
            #   ヘッダー行をインデックス: {header_row_index} で見つけました
            break
            
    if header_row_index is None:
        print(f"  [ERROR] No header row (with 'Location') found for GID {gid}.")
        #   [エラー] GID {gid} のヘッダー行（'Location'を含む）が見つかりません。
        return None # Skip this DataFrame

    # --- 3. Create the new, clean DataFrame ---
    # --- 3. 新しいクリーンなDataFrameの作成 ---
    
    # Set the new header
    # 新しいヘッダーを設定
    new_header = df.iloc[header_row_index]
    clean_df = df[header_row_index + 1:].copy() # Get all data below the header
    clean_df.columns = new_header
    
    # Clean column names (e.g., "Available Spaces 8:00 AM" -> "Available_Spaces")
    # カラム名をクリーンアップ（例：「Available Spaces 8:00 AM」->「Available_Spaces」）
    clean_df.columns = clean_df.columns.str.strip() # Remove whitespace
    clean_df.rename(columns=lambda x: re.sub(r'\s\d{1,2}:\d{2}\s[AP]M', '', x), inplace=True)
    clean_df.columns = clean_df.columns.str.replace(' ', '_')
    
    # Add the extracted metadata
    # 抽出したメタデータを追加
    clean_df['Date'] = date_str
    clean_df['Time_of_Reading'] = time_str
    clean_df['Source_GID'] = gid
    
    # Drop empty rows (where 'Location' is NaN)
    # 空の行（'Location'がNaN）を削除
    clean_df.dropna(subset=['Location'], inplace=True)
    
    # --- 4. Final Polish ---
    # --- 4. 最終仕上げ ---
    
    # Convert types
    # 型を変換
    if 'Available_Spaces' in clean_df.columns:
         clean_df['Available_Spaces'] = pd.to_numeric(clean_df['Available_Spaces'], errors='coerce')
    clean_df['Date'] = pd.to_datetime(clean_df['Date'], errors='coerce')
    
    print(f"  [SUCCESS] Cleaning complete for GID {gid}.")
    #   [成功] GID {gid} のクリーニングが完了しました。
    
    return clean_df

# --- Main part of the script ---
# --- スクリプトのメイン部分 ---

# This is the GID we want to test
# これがテストしたいGIDです
target_gid = '590671728'
# This is the *Sheet ID* from the URL
# これはURLからの*シートID*です
sheet_id = '1OscrXVHgOn75NrsrUxJfitmcPBHRj6xlnRcsq9jXo2g'

# Build the download URL
# ダウンロードURLを構築
download_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={target_gid}"

print(f"Downloading raw data for GID: {target_gid}...")
# GID: {target_gid} の生データをダウンロード中...

try:
    # 1. Download the raw data
    # 1. 生データをダウンロード
    response = requests.get(download_url)
    if response.status_code == 200:
        csv_content = response.content.decode('utf-8')
        csv_file = io.StringIO(csv_content)
        
        # Read with no header, just like in the full script
        # 完全なスクリプトと同様に、ヘッダーなしで読み込みます
        raw_df = pd.read_csv(csv_file, header=None)
        
        print("Raw data downloaded. Starting cleaning process...")
        # 生データをダウンロードしました。クリーニングプロセスを開始します...

        # 2. Call the cleaning function on this one DataFrame
        # 2. この1つのDataFrameに対してクリーニング関数を呼び出します
        cleaned_dataframe = clean_and_restructure_data(raw_df, target_gid)
        
        if cleaned_dataframe is not None:
            print("\n" + "="*50)
            print("--- Cleaned DataFrame (First 10 Rows) ---")
            # --- クリーンアップされたDataFrame（最初の10行）---
            print(cleaned_dataframe.head(10))
            
            print("\n--- Cleaned DataFrame Info ---")
            # --- クリーンアップされたDataFrameの情報 ---
            cleaned_dataframe.info()
        else:
            print("Cleaning failed.")
            # クリーニングに失敗しました。

    else:
        print(f"Failed to download. Status code: {response.status_code}")
        # ダウンロードに失敗しました。ステータスコード: {response.status_code}

except Exception as e:
    print(f"An error occurred: {e}")

Raw data downloaded. Starting cleaning process...
--- Cleaning GID 590671728 ---
  Found Date: Thursday, September 25, 2025
  Found Time: 8:00 AM
  Found Header at row index: 2
An error occurred: expected string or bytes-like object, got 'float'


In [ ]:
import pandas as pd
import requests
import io
import time
from bs4 import BeautifulSoup # You'll need to install this: pip install beautifulsoup4
import re
import os # <-- Added for saving files

# --- 1. Scrape the main page to find all the Google Sheet links ---

main_url = "https://transportation.ucsd.edu/commute/permits/availability.html"

# This dictionary will hold all our data
# We'll now store (csv_url, filename)
csv_links = {}

print(f"Fetching main page: {main_url}")

try:
    main_page_response = requests.get(main_url)
    
    if main_page_response.status_code == 200:
        print("Successfully fetched main page.")
        
        soup = BeautifulSoup(main_page_response.content, 'html.parser')
        
        iframes = soup.find_all('iframe')
        
        print(f"Found {len(iframes)} <iframe> tags. Filtering for Google Sheets...")

        for frame in iframes:
            src_link = frame.get('src')
            
            if src_link and 'docs.google.com/spreadsheets' in src_link and '/pubhtml' in src_link:
                
                # --- 2. Convert the /pubhtml link to a /export?format=csv link ---
                
                sheet_id_match = re.search(r'/d/([a-zA-Z0-9-_]+)', src_link)
                gid_match = re.search(r'gid=([0-9]+)', src_link)
                
                if sheet_id_match and gid_match:
                    sheet_id = sheet_id_match.group(1)
                    gid = gid_match.group(1)
                    
                    csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid}"
                    
                    # --- NEW: Find Week and Day for filename ---
                    filename = f"GID_{gid}.csv" # Default name
                    try:
                        # Climb up to the 'drawer' div to find the day
                        day_drawer = frame.find_parent('div', class_='drawer')
                        day_heading = day_drawer.find('h2', class_='expand')
                        day_name = day_heading.get_text(strip=True)
                        
                        # Climb up to the 'drawer-wrapper' to find the week
                        drawer_wrapper = day_drawer.find_parent('div', class_='drawer-wrapper')
                        week_heading = drawer_wrapper.find_previous_sibling('h2')
                        week_name = week_heading.get_text(strip=True)

                        # Sanitize names for a clean filename
                        safe_week = re.sub(r'\W+', '_', week_name)
                        safe_day = re.sub(r'\W+', '_', day_name)
                        filename = f"{safe_week}-{safe_day}.csv"
                        
                        print(f"  Found file: {filename}")

                    except Exception as e:
                        print(f"  Warning: Could not find Week/Day for GID {gid}. Using default name.")
                    
                    # Add it to our list, using the GID as a key to avoid duplicates
                    if gid not in csv_links:
                        csv_links[gid] = (csv_url, filename)

        print(f"Found {len(csv_links)} unique Google Sheet tabs to download.")

        # --- 3. Loop, Download, and SAVE each CSV ---
        files_saved_count = 0
        for gid, (download_url, filename) in csv_links.items():
            
            print(f"\nDownloading: {filename} (GID: {gid})")
            
            try:
                response = requests.get(download_url)
                if response.status_code == 200:
                    
                    # --- THIS IS THE NEW FILE WRITING PART ---
                    # We save the raw content, not a pandas DataFrame,
                    # as this is more reliable.
                    with open(filename, 'wb') as f:
                        f.write(response.content)
                    
                    print(f"  [SUCCESS] Saved file: {filename}")
                    files_saved_count += 1
                    
                else:
                    print(f"  [FAILED] Could not download {filename}. Status: {response.status_code}")
            
            except Exception as e:
                print(f"  [FAILED] An error occurred for {filename}: {e}")
            
            time.sleep(1) # Be polite

    else:
        print(f"Failed to fetch main page. Status code: {main_page_response.status_code}")

except Exception as e:
    print(f"An error occurred: {e}")

print("\n" + "="*50)
print("Scraping complete.")
print(f"Successfully saved {files_saved_count} files.")
print("The files are in the same directory as this script.")

Fetching main page: https://transportation.ucsd.edu/commute/permits/availability.html
Successfully fetched main page.
Found 12 <iframe> tags. Filtering for Google Sheets...
Found 12 unique Google Sheet tabs to download.

Downloading: GID_590671728.csv (GID: 590671728)
  [SUCCESS] Saved file: GID_590671728.csv

Downloading: GID_1596814157.csv (GID: 1596814157)
  [SUCCESS] Saved file: GID_1596814157.csv

Downloading: GID_0.csv (GID: 0)
  [SUCCESS] Saved file: GID_0.csv

Downloading: GID_1732410273.csv (GID: 1732410273)
  [SUCCESS] Saved file: GID_1732410273.csv

Downloading: GID_980526681.csv (GID: 980526681)
  [SUCCESS] Saved file: GID_980526681.csv

Downloading: GID_1691861364.csv (GID: 1691861364)
  [SUCCESS] Saved file: GID_1691861364.csv

Downloading: GID_1701491299.csv (GID: 1701491299)
  [SUCCESS] Saved file: GID_1701491299.csv

Downloading: GID_1997138837.csv (GID: 1997138837)
  [SUCCESS] Saved file: GID_1997138837.csv

Downloading: GID_1843964475.csv (GID: 1843964475)
  [SUCCESS

In [12]:
import os
import re
import glob

print("Starting file renaming process...")

# Find all files starting with 'GID_' and ending with '.csv'
# in the current directory
files_to_rename = glob.glob('GID_*.csv')

if not files_to_rename:
    print("No 'GID_*.csv' files found in this directory.")
    print("Please make sure this script is in the same folder as your CSV files.")
else:
    print(f"Found {len(files_to_rename)} files to rename.")

renamed_count = 0
for old_filename in files_to_rename:
    try:
        with open(old_filename, 'r') as f:
            # Read the first two lines which contain the info
            line1 = f.readline()
            line2 = f.readline()
        
        # --- Extract Information ---
        
        # From Line 1: "Parking Occupancy : Fall 2025 Week 2,,,,,,"
        week_match = re.search(r'Week (\d+)', line1)
        
        # From Line 2: ""Wednesday, October 8, 2025",,,Available Spaces,,,"
        day_match = re.search(r'"(\w+),', line2)
        
        if week_match and day_match:
            week_number = week_match.group(1)
            day_name = day_match.group(1)
            
            # --- Construct New Filename ---
            new_filename = f"Parking_Occupancy_Fall_2025_Week{week_number}_{day_name}.csv"
            
            # --- Rename the File ---
            os.rename(old_filename, new_filename)
            print(f"  Renamed: {old_filename}  ->  {new_filename}")
            renamed_count += 1
        else:
            print(f"  Skipping: {old_filename} (Could not find Week/Day info)")
            
    except Exception as e:
        print(f"  Error renaming {old_filename}: {e}")

print(f"\nRenaming complete. Successfully renamed {renamed_count} files.")

Starting file renaming process...
Found 10 files to rename.
  Renamed: GID_980526681.csv  ->  Parking_Occupancy_Fall_2025_Week1_Wednesday.csv
  Renamed: GID_1732410273.csv  ->  Parking_Occupancy_Fall_2025_Week1_Tuesday.csv
  Renamed: GID_1691861364.csv  ->  Parking_Occupancy_Fall_2025_Week1_Thursday.csv
  Renamed: GID_0.csv  ->  Parking_Occupancy_Fall_2025_Week1_Monday.csv
  Renamed: GID_1701491299.csv  ->  Parking_Occupancy_Fall_2025_Week1_Friday.csv
  Renamed: GID_1596814157.csv  ->  Parking_Occupancy_Fall_2025_Week0_Friday.csv
  Renamed: GID_590671728.csv  ->  Parking_Occupancy_Fall_2025_Week0_Thursday.csv
  Renamed: GID_1997138837.csv  ->  Parking_Occupancy_Fall_2025_Week2_Monday.csv
  Renamed: GID_871989841.csv  ->  Parking_Occupancy_Fall_2025_Week2_Wednesday.csv
  Renamed: GID_1843964475.csv  ->  Parking_Occupancy_Fall_2025_Week2_Tuesday.csv

Renaming complete. Successfully renamed 10 files.
